# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**План работы**
- обзор и предобработка данных
- категоризация данных
- исследовательский анализ данных
    - анализ зависимости возврата кредита от количества детей
    - анализ влияния семейного положение на возвращаемость кредитов


## Общая информация

In [1]:
import pandas as pd # импорт библиотеки pandas
df = pd.read_csv('/datasets/data.csv') # прочитаем файл 'data.csv' и сохраним его в переменной df
df.info() #просмотр сводной информации о таблице


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Согласно документации к данным:
- **children** — количество детей в семье
- **days_employed** — общий трудовой стаж в днях
- **dob_years** — возраст клиента в годах
- **education** — уровень образования клиента
- **education_id** — идентификатор уровня образования
- **family_status** — семейное положение
- **family_status_id** — идентификатор семейного положения
- **gender** — пол клиента
- **income_type** — тип занятости
- **debt** — имел ли задолженность по возврату кредитов
- **total_income** — ежемесячный доход
- **purpose** — цель получения кредита

In [2]:
df.head() #вывод первых строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


**Вывод**

В каждой строке таблицы - данные клиента. Часть колонок содержит личные данные и данные о семейном положении, другие - информацию о занятости и кредитной истории.
Количество значений в столбцах различается. Значит, в данных есть пропущенные значения. В столбце с информацией о трудовом стаже есть артефакты - отрицательные значения. 

## Предобработка данных

### Обработка пропусков

Избавимся от пропусков

In [3]:
df.isna().sum() #подсчет пропущенных значений

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

Проверим пропущены ли значения двух столбцов одновременно в одной строке

In [4]:
df[(df['days_employed'].isna()) & (~df['total_income'].isna())].count() 
#подсчет количества строк, где значение пропущено либо в столбце 'days_employed', либо в столбце 'total_income'

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Значения пропущены в одних и тех же строках, значит в анкетах 2174 клиентов отсутствует информация и трудовом стаже и доходах. Предположим, что это неработающие клиенты.

In [5]:
df.loc[df['days_employed'].isna()]['income_type'].value_counts()
#вывод уникальных значений столбца 'income_type' с подсчетом строк, где есть пропуски в столбце 'days_employed'

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64

Пропуски в информации о трудовом стаже и доходах не относятся к безработным клиентам. 
Маловероятно, что 2174 клиента не заполнили одни и те же данные, значит пропуски допущены по техническим причинам. Так как нет возможности установить причину пропусков и восстановить данные, напишем функцию, которая заменит пропуски медианными значениями стажа и дохода. Чтобы заполнить пропуски с группировкой по виду занятости используем метод `loc`:

In [6]:
df['days_employed'] = df['days_employed'].apply(abs) #модуль значений столбца 'days_employed'

def values_by_group(data, column, group):#функция для заполнения пропущенных значений медианой с группировкой по одной переменной
    for elem in data[group].unique(): #цикл перебирает уникальные значения заданного столбца
        median = data.loc[data[group] == elem, column].median() #находит медиану для каждого значения в заданной колонке
        data.loc[(data[column].isna()) & (data[group] == elem), column] = median #заполняет пропуски медианой


values_by_group(df, 'days_employed', 'income_type')
values_by_group(df, 'total_income', 'income_type')


In [7]:
df.isna().sum() #подсчет пропущенных значений после применения функции

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

**Вывод**

На основе обработки пропусков можно сделать вывод, что пропуски были допущены по техническим причинам. Всего было выявлено 2174 строки с пропусками, что составляет 10% от всей таблицы. Удаление такой значительной доли могло негативно повлиять на результаты исследования, поэтому пропуски были заполнены медианным значением.

### Замена типа данных

Для выявления артефактов проверим максимальные и минимальные значения возраста, стажа и количества детей

In [8]:
print('Возраст max:', df['dob_years'].max())
print('Возраст min:', df['dob_years'].min())
print('Стаж max:', df['days_employed'].max())
print('Стаж min:', df['days_employed'].min())
print('Дети max:', df['children'].max())
print('Дети min:', df['children'].min())


Возраст max: 75
Возраст min: 0
Стаж max: 401755.40047533
Стаж min: 24.14163324048118
Дети max: 20
Дети min: -1


Для более точного результата исследования избавимся от нулевых значений в колонке `dob_years`, аномальных значений в колонках `days_employed` и `children`.
Так как значения в столбце `dob_years` не влияют на результаты исследования, заменим нулевые значения на среднее значение по столбцу `dob_years`:

In [9]:
df.loc[df['dob_years'] == 0, 'dob_years'] = df['dob_years'].mean() 
#замена нулевых значений на среднее значение по столбцу 'dob_years'.
print('Возраст max:', df['dob_years'].max()) #проверка результата замены
print('Возраст min:', df['dob_years'].min())

Возраст max: 75.0
Возраст min: 19.0


In [10]:
df['children'] = df['children'].apply(abs) #модуль значений столбца 'children'
df[df['children'] == 20]['children'].count() #подсчет количества строк со значением "20" в данных о детях


76

Так как количество строк с аномальными значениями незначительно, заменим значения "20" в столбце `children` на среднее значение по столбцу:

In [11]:
df.loc[df['children'] == 20, 'children'] = df['children'].mean() 

print('Дети min:', df['children'].min()) #проверка результата замены
print('Дети max:', df['children'].max())

Дети min: 0.0
Дети max: 5.0


Предположим, что данные были собраны из разных источников и в каком-то стаж выражен в часах. Предположим, что максимальный стаж в днях 21900. Переведем аномальные значения стажа в дни.

In [12]:
df.loc[df['days_employed'] >= 21900, 'days_employed'] = df.loc[df['days_employed'] >= 21900, 'days_employed'] / 24 
#перевод стажа "в часах" в стаж "в днях" в строках, где значения стажа превышают 21900 дней
print('Стаж max:', df['days_employed'].max()) #проверка результата перевода
print('Стаж min:', df['days_employed'].min())

Стаж max: 18388.949900568383
Стаж min: 24.14163324048118


Заменим вещественный тип данных на целочисленный с помощью метода `astype()`.

In [13]:
df['children'] = df['children'].astype('int') #перевод значений столбца в целочисленный вид
df['days_employed'] = df['days_employed'].astype('int') #перевод значений столбца в целочисленный вид
df['dob_years'] = df['dob_years'].astype('int') #перевод значений столбца в целочисленный вид
df['total_income'] = df['total_income'].astype('int') #перевод значений столбца в целочисленный вид


df.info() #проверка результата


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   children          21525 non-null  int64 
 1   days_employed     21525 non-null  int64 
 2   dob_years         21525 non-null  int64 
 3   education         21525 non-null  object
 4   education_id      21525 non-null  int64 
 5   family_status     21525 non-null  object
 6   family_status_id  21525 non-null  int64 
 7   gender            21525 non-null  object
 8   income_type       21525 non-null  object
 9   debt              21525 non-null  int64 
 10  total_income      21525 non-null  int64 
 11  purpose           21525 non-null  object
dtypes: int64(7), object(5)
memory usage: 2.0+ MB


**Вывод**

Данные содержали аномальные значения, не подходящие для анализа. Для удобства расчетов были исключены артефакты и вещественный тип данных был заменен на целочисленный. 

### Обработка дубликатов

In [14]:
df.duplicated().sum() #подсчет явных дубликатов

54

Чтобы учесть возможные дубликаты с разным регистром, найдем уникальные значения столбца с количеством их упоминаний:

In [15]:
print('Цель получения кредита:')

print(df['purpose'].value_counts())

print()
print('Семейное положение:')

print(df['family_status'].value_counts())
print()
print('Уровень образования клиента:')

print(df['education'].value_counts())


Цель получения кредита:
свадьба                                   797
на проведение свадьбы                     777
сыграть свадьбу                           774
операции с недвижимостью                  676
покупка коммерческой недвижимости         664
покупка жилья для сдачи                   653
операции с жильем                         653
операции с коммерческой недвижимостью     651
жилье                                     647
покупка жилья                             647
покупка жилья для семьи                   641
строительство собственной недвижимости    635
недвижимость                              634
операции со своей недвижимостью           630
строительство жилой недвижимости          626
покупка недвижимости                      624
покупка своего жилья                      620
строительство недвижимости                620
ремонт жилью                              612
покупка жилой недвижимости                607
на покупку своего автомобиля              505
заняться в

В столбце `education` есть одинаковые значения  с разным регистром. Приведем все значения столбца `education` к нижнему регистру с помощью метода `str.lower()`

In [16]:
df['education'] = df['education'].str.lower()
df['education'].value_counts() #проверка результата

среднее                15233
высшее                  5260
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64

In [17]:
df.duplicated().sum() #подсчет явных дубликатов

71

In [18]:
df[df.duplicated(keep=False)].sort_values(by=list(df.columns)).head(10) #просмотр строк с дубликатами

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
9374,0,1547,38,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
19387,0,1547,38,высшее,0,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
5124,0,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,сыграть свадьбу
10697,0,1547,40,среднее,1,гражданский брак,1,F,компаньон,0,172357,сыграть свадьбу
16148,0,1547,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
19369,0,1547,45,среднее,1,гражданский брак,1,F,компаньон,0,172357,свадьба
9920,0,1547,51,среднее,1,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
15991,0,1547,51,среднее,1,гражданский брак,1,F,компаньон,0,172357,на проведение свадьбы
2254,0,1547,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью
17379,0,1547,54,высшее,0,женат / замужем,0,M,компаньон,0,172357,операции с коммерческой недвижимостью


In [19]:
df = df.drop_duplicates().reset_index(drop=True) #удаление явных дубликатов

**Вывод**

Дубликаты могли появиться в результате ошибки в заполнении анкеты, а также в результате объединения данных из разных источников. После приведения значений к единому регистру были найдены явные дубликаты, которые могли повлиять на расчеты. После удаления дубликатов данные готовы к исследованию.

### Лемматизация

В столбце "цель получения кредита" встречается много значений с одинаковой целью, записанной разными словами. Чтобы упростить анализ выделим основные группы целей получения кредита. Для этого лемматизируем значения столбца `purpose`.

In [21]:
from pymystem3 import Mystem #импорт библиотеки pymystem
m = Mystem()
lemmas = m.lemmatize(' '.join(df['purpose'].unique())) #собираем уникальные значения столбца 'purpose' в строку и лемматизируем
print(lemmas)

['покупка', ' ', 'жилье', ' ', 'приобретение', ' ', 'автомобиль', ' ', 'дополнительный', ' ', 'образование', ' ', 'сыграть', ' ', 'свадьба', ' ', 'операция', ' ', 'с', ' ', 'жилье', ' ', 'образование', ' ', 'на', ' ', 'проведение', ' ', 'свадьба', ' ', 'покупка', ' ', 'жилье', ' ', 'для', ' ', 'семья', ' ', 'покупка', ' ', 'недвижимость', ' ', 'покупка', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'покупка', ' ', 'жилой', ' ', 'недвижимость', ' ', 'строительство', ' ', 'собственный', ' ', 'недвижимость', ' ', 'недвижимость', ' ', 'строительство', ' ', 'недвижимость', ' ', 'на', ' ', 'покупка', ' ', 'подержать', ' ', 'автомобиль', ' ', 'на', ' ', 'покупка', ' ', 'свой', ' ', 'автомобиль', ' ', 'операция', ' ', 'с', ' ', 'коммерческий', ' ', 'недвижимость', ' ', 'строительство', ' ', 'жилой', ' ', 'недвижимость', ' ', 'жилье', ' ', 'операция', ' ', 'со', ' ', 'свой', ' ', 'недвижимость', ' ', 'автомобиль', ' ', 'заниматься', ' ', 'образование', ' ', 'сделка', ' ', 'с', ' ', 'подержать

Чтобы выделить основные группы, посчитаем число упоминаний каждого слова после лемматизации

In [22]:
from collections import Counter #вызов специального контейнера Counter из модуля collections

In [23]:
lemmas_not_unique = m.lemmatize(' '.join(df['purpose'])) #лемматизируем значения столбца 'purpose'
print(Counter(lemmas_not_unique)) 

Counter({' ': 55023, 'недвижимость': 6351, 'покупка': 5897, 'жилье': 4460, 'автомобиль': 4306, 'образование': 4013, 'с': 2918, 'операция': 2604, 'свадьба': 2324, 'свой': 2230, 'на': 2222, 'строительство': 1878, 'высокий': 1374, 'получение': 1314, 'коммерческий': 1311, 'для': 1289, 'жилой': 1230, 'сделка': 941, 'дополнительный': 906, 'заниматься': 904, 'подержать': 853, 'проведение': 768, 'сыграть': 765, 'сдача': 651, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'приобретение': 461, 'профильный': 436, 'подержанный': 111, '\n': 1})


На основе полученных данных выделим 4 основные группы: "недвижимость", "автомобиль", "образование", "свадьба". Напишем функцию, которая будет возвращать название группы, если это название встречается в лемматизированном значении заданного столбца.

In [24]:
def purpose_grouped(purpose):
    lemmas = m.lemmatize(purpose)
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    elif 'автомобиль' in lemmas:
        return 'автомобиль'
    elif 'образование' in lemmas:
        return 'образование'
    elif 'свадьба' in lemmas:
        return 'свадьба'
    return 'прочее'
    
df['purpose'] = df['purpose'].apply(purpose_grouped) #замена значений в столбце 'purpose' на значения после применения функции
df['purpose'].value_counts() #вывод новых значений стобца 'purpose' с количеством их упоминаний


недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

**Вывод**

После лемматизации было выделено 4 основные группы целей получения кредита, что значительно упрощает дальнейший анализ и формирование вывода.

### Категоризация данных

Для проверки платежеспомобности отдельных категорий клиентов, выделим те категории, которые интересуют заказчика. Категоризируем клиентов по количеству детей, уровню дохода, семейному положению и цели получения кредита.
Выделим категории и посчитаем количество клиентов в каждой категории по семейному положению:

In [25]:
df['family_status'].value_counts() #вывод уникальных значений стобца 'family_status' с количеством их упоминаний

женат / замужем          12339
гражданский брак          4151
Не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64

Выделим категории и посчитаем количество клиентов в каждой категории по наличию детей. 

In [26]:
def child_or_not(row):
    if row == 0:
        return 'есть дети'
    return 'нет детей'
    
df['child_id'] = df['children'].apply(child_or_not) 
#создаем новый столбец со значениями 'есть дети' и 'нет детей'
df['child_id'].value_counts()

есть дети    14167
нет детей     7287
Name: child_id, dtype: int64

Выделим категории и посчитаем количество клиентов в каждой категории по количеству детей:

In [27]:
df['children'].value_counts() #вывод уникальных значений стобца 'children' с количеством их упоминаний

0    14167
1     4855
2     2052
3      330
4       41
5        9
Name: children, dtype: int64

Выделим категории и посчитаем количество клиентов в каждой категории по цели получения кредита:

In [28]:
df['purpose'].value_counts() #вывод уникальных значений стобца 'purpose' с количеством их упоминаний

недвижимость    10811
автомобиль       4306
образование      4013
свадьба          2324
Name: purpose, dtype: int64

Для категоризации по уровню дохода выделим три равные группы с помощью функции `qcut()`:

In [29]:
df['income_category'] = pd.qcut(df['total_income'], 3, labels=["ниже среднего", "средний", "высокий"])
df['income_category'].value_counts()

средний          7285
ниже среднего    7152
высокий          7017
Name: income_category, dtype: int64

In [30]:
df.head() #вывод первых строк таблицы

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,child_id,income_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,недвижимость,нет детей,высокий
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,автомобиль,нет детей,ниже среднего
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,недвижимость,есть дети,средний
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,образование,нет детей,высокий
4,0,14177,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,свадьба,есть дети,средний


**Вывод**

В результате предобработки данных были устранены пропущенные значения и дубликаты, данные не содержат аномальных значений. В результате категоризации были выделены только актуальные данные, после чего можно перейти к формированию выводов.

## Анализ зависимости

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [31]:
report = df.pivot_table(index=['child_id'], values='debt', aggfunc=['sum', 'count', 'mean']) 
#формируем сводную таблицу с помощью функции pivot_table() по наличию детей, с подсчетом значений в столбце 'debt'
report.columns = ['к-во случаев невозврата', 'общее к-во заемщиков', '% невозврата'] #переименуем столбцы
report.index = report.index.rename('дети')#переименуем столбец группировки
report = report.sort_values(by='% невозврата', ascending=False) #сортировка таблицы по столбцу '% невозврата' по убыванию
report = report.style.format({'% невозврата': '{:.2%}'}) #изменим формат вывода значений столбца '% невозврата'
report

,к-во случаев невозврата,общее к-во заемщиков,% невозврата
дети,,,
нет детей,670,7287,9.19%
есть дети,1071,14167,7.56%


Проверим влияет ли количество детей на возврат кредита в срок

In [32]:
report = df.pivot_table(index=['children'], values='debt', aggfunc=['sum', 'count', 'mean']) 
#формируем сводную таблицу с помощью функции pivot_table() по количеству детей, с подсчетом значений в столбце 'debt'
report.columns = ['к-во случаев невозврата', 'общее к-во заемщиков', '% невозврата'] #переименуем столбцы
report.index = report.index.rename('к-во детей')#переименуем столбец группировки
report = report.sort_values(by='% невозврата', ascending=False) #сортировка таблицы по столбцу '% невозврата' по убыванию
report = report.style.format({'% невозврата': '{:.2%}'}) #изменим формат вывода значений столбца '% невозврата'
report

,к-во случаев невозврата,общее к-во заемщиков,% невозврата
к-во детей,,,
4,4,41,9.76%
2,194,2052,9.45%
1,445,4855,9.17%
3,27,330,8.18%
0,1071,14167,7.56%
5,0,9,0.00%


Как видно из таблиц, клиенты, у которых нет, детей реже имеют задолженность по кредиту. В то же время количество детей значительно не влияет на возврат. Не будем брать во внимание 100% возврат клиентов с пятью детьми, так как данных слишком мало для сравнения

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [33]:
report = df.groupby('family_status').agg({'debt': ['sum', 'count', 'mean']})
report.columns = ['к-во случаев невозврата', 'общее к-во заемщиков', '% невозврата']#переименуем столбцы
report.index = report.index.rename('семейное положение')#переименуем столбец группировки
report = report.sort_values(by='% невозврата', ascending=False)#сортировка таблицы по столбцу '% невозврата' по убыванию
report = report.style.format({'% невозврата': '{:.2%}'})#изменим формат вывода значений столбца '% невозврата'
report

,к-во случаев невозврата,общее к-во заемщиков,% невозврата
семейное положение,,,
Не женат / не замужем,274,2810,9.75%
гражданский брак,388,4151,9.35%
женат / замужем,931,12339,7.55%
в разводе,85,1195,7.11%
вдовец / вдова,63,959,6.57%


На основе полученных данных можно сделать вывод, что люди, не состоящие в браке, менее заинтересованы в возврате кредита, а клиенты, которые развелись или овдовели чаще погашают кредит вовремя, чем те, кто состоит в браке.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [34]:
report = df.groupby('income_category').agg({'debt': ['sum', 'count', 'mean']})
report.columns = ['к-во случаев невозврата', 'общее к-во заемщиков', '% невозврата']#переименуем столбцы
report.index = report.index.rename('уровень дохода')#переименуем столбец группировки
report = report.sort_values(by='% невозврата', ascending=False)#сортировка таблицы по столбцу '% невозврата' по убыванию
report = report.style.format({'% невозврата': '{:.2%}'})#изменим формат вывода значений столбца '% невозврата'
report

,к-во случаев невозврата,общее к-во заемщиков,% невозврата
уровень дохода,,,
средний,632,7285,8.68%
ниже среднего,581,7152,8.12%
высокий,528,7017,7.52%


Самый низкий процент невозврата у клиентов с высоким доходом, а клиенты с доходом ниже среднего более надежные заемщики, чем клиенты со средним доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [35]:
report = df.groupby('purpose').agg({'debt': ['sum', 'count', 'mean']})
report.columns = ['к-во случаев невозврата', 'общее к-во заемщиков', '% невозврата']#переименуем столбцы
report.index = report.index.rename('цель кредитования')#переименуем столбец группировки
report = report.sort_values(by='% невозврата', ascending=False)#сортировка таблицы по столбцу '% невозврата' по убыванию
report = report.style.format({'% невозврата': '{:.2%}'})#изменим формат вывода значений столбца '% невозврата'
report

,к-во случаев невозврата,общее к-во заемщиков,% невозврата
цель кредитования,,,
автомобиль,403,4306,9.36%
образование,370,4013,9.22%
свадьба,186,2324,8.00%
недвижимость,782,10811,7.23%


Как видно из таблицы, меньше просроченной задолженности у клиентов, которые берут кредит на свадьбу или приобретение недвижимости. Самый высокий процент невозврата у клиентов, кто приобретает автомобиль и чуть меньше у тех, кто оплачивает образование.

## Общий вывод

Вероятность возврата кредита в срок зависит от семейного положения - люди, никогда не состоявшие в браке, чаще имеют просроченную задолженность. Процент невозврата у заемщиков состоящих в браке на 2% ниже. Клиенты, не имеющие детей, чаще погашают кредит в срок, а количество детей сильно не влияет на процент невозврата. Так, наличие ребенка в семье снижает надежность клиента на 1,63%. Также стоит обратить внимание на уровень дохода клиента и цель получения кредита. Например, надежность заемщика, не состоящего в браке, может укрепить высокий доход.